# Measuring the speed for all of page detectors and screen detectors

In [14]:
from datetime import datetime

from video699.__main__ import PAGE_DETECTOR_NAMES, SCREEN_DETECTOR_NAMES

In [15]:
def duration(page_detector_name, screen_detector_name):
    start_time = datetime.now()
    ! TF_CPP_MIN_LOG_LEVEL=2 python -m video699 \
        --institution 'fimu' \
        --room 'd2' \
        --camera 'default_2004' \
        --date '2019-11-12T12:00:00+01:00' \
        --documents 'Blockchain-FI-MUNI.pdf' \
        --video 'IA067-D2-20191112.mp4' \
        --output '/dev/null' \
        --page-detector '{page_detector_name}' \
        --screen-detector '{screen_detector_name}' \
        --scene-detector 'none'
    finish_time = datetime.now()
    duration = finish_time - start_time
    return duration

In [17]:
durations = {}
for page_detector_name in PAGE_DETECTOR_NAMES:
    durations[page_detector_name] = dict()
    for screen_detector_name in SCREEN_DETECTOR_NAMES:
        durations[page_detector_name][screen_detector_name] = duration(page_detector_name, screen_detector_name)
        
        print(
            'durations[{}][{}] = {}'.format(
                page_detector_name,
                screen_detector_name,
                durations[page_detector_name][screen_detector_name].total_seconds(),
            )
        )

Using TensorFlow backend.
Reading IA067-D2-20191112.mp4: frame 50541, time 0:56:09.266667, speed 0.80x

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Reading IA067-D2-20191112.mp4: frame 68178, time 1:15:45.066667, speed 0.80x
Exception ignored in: <function Handle.__del__ at 0x7f790dfaa4d0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
  File "/opt/conda/lib/python3.7/site-packages/rtree/index.py", line 864, in destroy
  File "/opt/conda/lib/python3.7/site-packages/rtree/core.py", line 34, in check_void_done
AttributeError: 'NoneType' object has no attribute 'Error_GetErrorCount'
Exception ignored in: <function Handle.__del__ at 0x7f790dfaa4d0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
  File "/opt/conda/lib/python3.7/site-packages/rtree/index.py", line 864, in destroy
  File "/opt/conda/lib/python3.7/site-packages/rtree/core.py", line 34, in check_void_done
AttributeError: 'NoneType' object has no attribute 'Error_GetErrorCount'
durations[siamese][fastai] = 5688.935913
Using Tensor

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Reading IA067-D2-20191112.mp4: frame 37453, time 0:41:36.733333, speed 0.89x

...

Reading IA067-D2-20191112.mp4: frame 43531, time 0:48:21.933333, speed 0.89x

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Reading IA067-D2-20191112.mp4: frame 49294, time 0:54:46.133333, speed 0.89x

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Reading IA067-D2-20191112.mp4: frame 68178, time 1:15:45.066667, speed 0.89x
durations[imagehash][fastai] = 5119.43368
Reading IA067-D2-20191112.mp4: frame 68178, time 1:15:45.066667, speed 10.31x
durations[imagehash][annotated] = 442.221636
Using TensorFlow backend.
Reading IA067-D2-20191112.mp4: frame 68178, time 1:15:45.066667, speed 0.71x
Exception ignored in: <function Handle.__del__ at 0x7f38222834d0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
  File "/opt/conda/lib/python3.7/site-packages/rtree/index.py", line 864, in destroy
  File "/opt/conda/lib/python3.7/site-packages/rtree/core.py", line 34, in check_void_done
AttributeError: 'NoneType' object has no attribute 'Error_GetErrorCount'
Exception ignored in: <function Handle.__del__ at 0x7f38222834d0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
  File "/opt/conda/lib/python3.7/s